In [25]:
library(biomaRt)
library(DESeq2)
library(tidyverse)
library(recount3)

In [2]:
mart <- useDataset("hsapiens_gene_ensembl", useMart("ensembl"))

In [3]:
human_projects <- available_projects()

2021-10-01 08:20:55 caching file sra.recount_project.MD.gz.

2021-10-01 08:20:55 caching file gtex.recount_project.MD.gz.

2021-10-01 08:20:56 caching file tcga.recount_project.MD.gz.



In [58]:
human_projects %>%
    filter(file_source == "sra", organism == "human") %>%
    select(project, file_source, n_samples) %>%
    arrange(desc(n_samples))

project,file_source,n_samples
<chr>,<chr>,<table>
SRP166108,sra,8464
ERP107748,sra,7598
SRP066781,sra,3854
SRP079058,sra,3589
ERP017126,sra,3514
ERP108653,sra,2687
SRP067759,sra,2289
SRP132726,sra,2260
SRP013565,sra,2212


In [4]:
human_projects %>%
    filter(file_source != "sra", organism == "human") %>%
    select(project, file_source, n_samples) %>%
    arrange(desc(n_samples))

project,file_source,n_samples
<chr>,<chr>,<table>
BRAIN,gtex,2931
SKIN,gtex,1940
ESOPHAGUS,gtex,1577
BLOOD_VESSEL,gtex,1398
ADIPOSE_TISSUE,gtex,1293
BRCA,tcga,1256
BLOOD,gtex,1048
HEART,gtex,942
MUSCLE,gtex,881


In [51]:
proj_tcga_gbm <- subset(
    human_projects,
    project == "GBM" & project_type == "data_sources"
)
proj_tcga_gbm

,project,organism,file_source,project_home,project_type,n_samples
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
8718,GBM,human,tcga,data_sources/tcga,data_sources,175


In [52]:
rse_gene_tcga_gbm <- create_rse(proj_tcga_gbm) 

2021-10-01 10:57:39 downloading and reading the metadata.

2021-10-01 10:57:40 caching file tcga.tcga.GBM.MD.gz.

adding rname 'http://duffel.rail.bio/recount3/human/data_sources/tcga/metadata/BM/GBM/tcga.tcga.GBM.MD.gz'

2021-10-01 10:57:41 caching file tcga.recount_project.GBM.MD.gz.

adding rname 'http://duffel.rail.bio/recount3/human/data_sources/tcga/metadata/BM/GBM/tcga.recount_project.GBM.MD.gz'

2021-10-01 10:57:42 caching file tcga.recount_qc.GBM.MD.gz.

adding rname 'http://duffel.rail.bio/recount3/human/data_sources/tcga/metadata/BM/GBM/tcga.recount_qc.GBM.MD.gz'

2021-10-01 10:57:43 caching file tcga.recount_seq_qc.GBM.MD.gz.

adding rname 'http://duffel.rail.bio/recount3/human/data_sources/tcga/metadata/BM/GBM/tcga.recount_seq_qc.GBM.MD.gz'

2021-10-01 10:57:44 downloading and reading the feature information.

2021-10-01 10:57:45 caching file human.gene_sums.G026.gtf.gz.

2021-10-01 10:57:45 downloading and reading the counts: 175 samples across 63856 features.

2021-10-01

In [59]:
proj_sra_SRP166108 <- subset(
    human_projects,
    project == "SRP166108" & project_type == "data_sources"
)
proj_sra_SRP166108

,project,organism,file_source,project_home,project_type,n_samples
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
3068,SRP166108,human,sra,data_sources/sra,data_sources,8464


In [60]:
rse_gene_sra_SRP166108 <- create_rse(proj_sra_SRP166108)

2021-10-01 11:10:18 downloading and reading the metadata.

2021-10-01 11:10:20 caching file sra.sra.SRP166108.MD.gz.

adding rname 'http://duffel.rail.bio/recount3/human/data_sources/sra/metadata/08/SRP166108/sra.sra.SRP166108.MD.gz'

2021-10-01 11:10:22 caching file sra.recount_project.SRP166108.MD.gz.

adding rname 'http://duffel.rail.bio/recount3/human/data_sources/sra/metadata/08/SRP166108/sra.recount_project.SRP166108.MD.gz'

2021-10-01 11:10:24 caching file sra.recount_qc.SRP166108.MD.gz.

adding rname 'http://duffel.rail.bio/recount3/human/data_sources/sra/metadata/08/SRP166108/sra.recount_qc.SRP166108.MD.gz'

2021-10-01 11:10:26 caching file sra.recount_seq_qc.SRP166108.MD.gz.

adding rname 'http://duffel.rail.bio/recount3/human/data_sources/sra/metadata/08/SRP166108/sra.recount_seq_qc.SRP166108.MD.gz'

2021-10-01 11:10:28 caching file sra.recount_pred.SRP166108.MD.gz.

adding rname 'http://duffel.rail.bio/recount3/human/data_sources/sra/metadata/08/SRP166108/sra.recount_pred.S

In [48]:
proj_gtex_blood <- subset(
    human_projects,
    project == "BLOOD" & project_type == "data_sources"
)
proj_gtex_blood

,project,organism,file_source,project_home,project_type,n_samples
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
8702,BLOOD,human,gtex,data_sources/gtex,data_sources,1048


In [49]:
rse_gene_gtex_blood <- create_rse(proj_gtex_blood)

2021-10-01 10:54:20 downloading and reading the metadata.

2021-10-01 10:54:21 caching file gtex.gtex.BLOOD.MD.gz.

2021-10-01 10:54:21 caching file gtex.recount_project.BLOOD.MD.gz.

2021-10-01 10:54:22 caching file gtex.recount_qc.BLOOD.MD.gz.

2021-10-01 10:54:22 caching file gtex.recount_seq_qc.BLOOD.MD.gz.

2021-10-01 10:54:23 downloading and reading the feature information.

2021-10-01 10:54:23 caching file human.gene_sums.G026.gtf.gz.

2021-10-01 10:54:24 downloading and reading the counts: 1048 samples across 63856 features.

2021-10-01 10:54:24 caching file gtex.gene_sums.BLOOD.G026.gz.

2021-10-01 10:54:46 construcing the RangedSummarizedExperiment (rse) object.



In [7]:
rse_gene_gtex_blood

class: RangedSummarizedExperiment 
dim: 63856 1048 
metadata(8): time_created recount3_version ... annotation recount3_url
assays(1): raw_counts
rownames(63856): ENSG00000278704.1 ENSG00000277400.1 ...
  ENSG00000182484.15_PAR_Y ENSG00000227159.8_PAR_Y
rowData names(10): source type ... havana_gene tag
colnames(1048): GTEX-R55C-0005-SM-3GAE9.1 GTEX-QMR6-0005-SM-32PKY.1 ...
  GTEX-1NSGN-0005-SM-DKPPX.1 GTEX-1QP29-0005-SM-DLZQX.1
colData names(198): rail_id external_id ... recount_seq_qc.errq
  BigWigURL

# Clinical

In [62]:
colData(rse_gene_sra_SRP166108) |> 
    as_tibble() %>%
    select(-sra.study_abstract) %>%
    head()

rail_id,external_id,study,sra.sample_acc.x,sra.experiment_acc,sra.submission_acc,sra.submission_center,sra.submission_lab,sra.study_title,sra.study_description,⋯,recount_seq_qc.avgq,recount_seq_qc.errq,recount_pred.sample_acc.y,recount_pred.curated.type,recount_pred.curated.tissue,recount_pred.pattern.predict.type,recount_pred.pred.type,recount_pred.curated.cell_type,recount_pred.curated.cell_line,BigWigURL
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,⋯,<dbl>,<dbl>,<chr>,<chr>,<lgl>,<chr>,<chr>,<lgl>,<lgl>,<chr>
2359359,SRR8087001,SRP166108,SRS3959220,SRX4913842,SRA797033,National Institute of Environmental Health Sciences,National Toxicology Program,The Power of Resolution: Contextualized Understanding of Chemical-biological Interactions,NA,⋯,36.4,29.5,SRS3959220,target-locus,NA,rna-seq,scrna-seq,NA,NA,http://duffel.rail.bio/recount3/human/data_sources/sra/base_sums/08/SRP166108/01/sra.base_sums.SRP166108_SRR8087001.ALL.bw
2360606,SRR8087028,SRP166108,SRS3959185,SRX4913808,SRA797033,National Institute of Environmental Health Sciences,National Toxicology Program,The Power of Resolution: Contextualized Understanding of Chemical-biological Interactions,NA,⋯,36.3,29.6,SRS3959185,target-locus,NA,rna-seq,scrna-seq,NA,NA,http://duffel.rail.bio/recount3/human/data_sources/sra/base_sums/08/SRP166108/28/sra.base_sums.SRP166108_SRR8087028.ALL.bw
2360607,SRR8095028,SRP166108,SRS3967208,SRX4921886,SRA797643,National Institute of Environmental Health Sciences,National Toxicology Program,The Power of Resolution: Contextualized Understanding of Chemical-biological Interactions,NA,⋯,36.4,29.9,SRS3967208,target-locus,NA,rna-seq,scrna-seq,NA,NA,http://duffel.rail.bio/recount3/human/data_sources/sra/base_sums/08/SRP166108/28/sra.base_sums.SRP166108_SRR8095028.ALL.bw
2362143,SRR8087058,SRP166108,SRS3959378,SRX4914000,SRA797033,National Institute of Environmental Health Sciences,National Toxicology Program,The Power of Resolution: Contextualized Understanding of Chemical-biological Interactions,NA,⋯,36.4,29.7,SRS3959378,target-locus,NA,rna-seq,scrna-seq,NA,NA,http://duffel.rail.bio/recount3/human/data_sources/sra/base_sums/08/SRP166108/58/sra.base_sums.SRP166108_SRR8087058.ALL.bw
2370079,SRR8081150,SRP166108,SRS3954450,SRX4908409,SRA796268,National Institute of Environmental Health Sciences,National Toxicology Program,The Power of Resolution: Contextualized Understanding of Chemical-biological Interactions,NA,⋯,36.5,30.0,SRS3954450,target-locus,NA,rna-seq,scrna-seq,NA,NA,http://duffel.rail.bio/recount3/human/data_sources/sra/base_sums/08/SRP166108/50/sra.base_sums.SRP166108_SRR8081150.ALL.bw
2370109,SRR8081151,SRP166108,SRS3954452,SRX4908411,SRA796268,National Institute of Environmental Health Sciences,National Toxicology Program,The Power of Resolution: Contextualized Understanding of Chemical-biological Interactions,NA,⋯,36.5,29.9,SRS3954452,target-locus,NA,rna-seq,scrna-seq,NA,NA,http://duffel.rail.bio/recount3/human/data_sources/sra/base_sums/08/SRP166108/51/sra.base_sums.SRP166108_SRR8081151.ALL.bw


In [56]:
colData(rse_gene_tcga_gbm) |> 
    as_tibble() %>%
    head()

rail_id,external_id,study,tcga.tcga_barcode,tcga.gdc_file_id,tcga.gdc_cases.project.name,tcga.gdc_cases.project.released,tcga.gdc_cases.project.state,tcga.gdc_cases.project.primary_site,tcga.gdc_cases.project.project_id,⋯,recount_seq_qc..distinct_quality_values,recount_seq_qc..bases,recount_seq_qc..a,recount_seq_qc..c,recount_seq_qc..g,recount_seq_qc..t,recount_seq_qc..n,recount_seq_qc.avgq,recount_seq_qc.errq,BigWigURL
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,⋯,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
106767,3b7f80e6-74a7-4da1-ac4b-c260b6be880a,GBM,TCGA-06-0649-01B-01R-1849-01,3b7f80e6-74a7-4da1-ac4b-c260b6be880a,Glioblastoma Multiforme,TRUE,legacy,Brain,TCGA-GBM,⋯,38,4812533332,26.0,25.6,23.9,24.4,0.0,30.7,11.5,http://duffel.rail.bio/recount3/human/data_sources/tcga/base_sums/BM/GBM/0A/tcga.base_sums.GBM_3b7f80e6-74a7-4da1-ac4b-c260b6be880a.ALL.bw
107190,97a8e2b1-059b-4527-8fb4-116f718ffd16,GBM,TCGA-14-1823-01A-01R-1849-01,97a8e2b1-059b-4527-8fb4-116f718ffd16,Glioblastoma Multiforme,TRUE,legacy,Brain,TCGA-GBM,⋯,38,5003989948,27.4,25.2,22.2,25.2,0.1,31.5,12.0,http://duffel.rail.bio/recount3/human/data_sources/tcga/base_sums/BM/GBM/16/tcga.base_sums.GBM_97a8e2b1-059b-4527-8fb4-116f718ffd16.ALL.bw
107292,bb96c5ba-ca50-40b4-b22a-3fcfe46ab11c,GBM,TCGA-28-5208-01A-01R-1850-01,bb96c5ba-ca50-40b4-b22a-3fcfe46ab11c,Glioblastoma Multiforme,TRUE,legacy,Brain,TCGA-GBM,⋯,38,5046398252,25.8,24.6,24.4,25.1,0.0,30.3,11.5,http://duffel.rail.bio/recount3/human/data_sources/tcga/base_sums/BM/GBM/1C/tcga.base_sums.GBM_bb96c5ba-ca50-40b4-b22a-3fcfe46ab11c.ALL.bw
108719,443b59bb-0bfd-459a-896e-fa07aa6c3a45,GBM,TCGA-06-0125-02A-11R-2005-01,443b59bb-0bfd-459a-896e-fa07aa6c3a45,Glioblastoma Multiforme,TRUE,legacy,Brain,TCGA-GBM,⋯,38,4742363444,24.8,25.4,25.0,24.8,0.0,31.8,12.5,http://duffel.rail.bio/recount3/human/data_sources/tcga/base_sums/BM/GBM/45/tcga.base_sums.GBM_443b59bb-0bfd-459a-896e-fa07aa6c3a45.ALL.bw
108733,633064db-d3ed-4b5f-b48f-6924ad719647,GBM,TCGA-19-2619-01A-01R-1850-01,633064db-d3ed-4b5f-b48f-6924ad719647,Glioblastoma Multiforme,TRUE,legacy,Brain,TCGA-GBM,⋯,38,5828310952,25.2,25.6,24.6,24.6,0.0,31.2,12.1,http://duffel.rail.bio/recount3/human/data_sources/tcga/base_sums/BM/GBM/47/tcga.base_sums.GBM_633064db-d3ed-4b5f-b48f-6924ad719647.ALL.bw
109845,4b93a447-e626-4040-9ba0-1eeab3764c6b,GBM,TCGA-41-3915-01A-01R-1850-01,4b93a447-e626-4040-9ba0-1eeab3764c6b,Glioblastoma Multiforme,TRUE,legacy,Brain,TCGA-GBM,⋯,38,5274691992,25.3,26.3,24.5,23.8,0.0,30.4,11.5,http://duffel.rail.bio/recount3/human/data_sources/tcga/base_sums/BM/GBM/6B/tcga.base_sums.GBM_4b93a447-e626-4040-9ba0-1eeab3764c6b.ALL.bw


In [57]:
colData(rse_gene_gtex_blood) |> 
    as_tibble() %>%
    head()

rail_id,external_id,study,gtex.run_acc,gtex.subjid,gtex.sex,gtex.age,gtex.dthhrdy,gtex.sampid,gtex.smatsscr,⋯,recount_seq_qc..distinct_quality_values,recount_seq_qc..bases,recount_seq_qc..a,recount_seq_qc..c,recount_seq_qc..g,recount_seq_qc..t,recount_seq_qc..n,recount_seq_qc.avgq,recount_seq_qc.errq,BigWigURL
<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<chr>,<lgl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
10044928,GTEX-R55C-0005-SM-3GAE9.1,BLOOD,SRR659319,GTEX-R55C,1,40-49,0,GTEX-R55C-0005-SM-3GAE9,NA,⋯,38,4129992836,23.5,26.4,27.4,22.7,0.0,35.2,16.3,http://duffel.rail.bio/recount3/human/data_sources/gtex/base_sums/OD/BLOOD/E9/gtex.base_sums.BLOOD_GTEX-R55C-0005-SM-3GAE9.1.ALL.bw
10044929,GTEX-QMR6-0005-SM-32PKY.1,BLOOD,SRR660579,GTEX-QMR6,1,50-59,4,GTEX-QMR6-0005-SM-32PKY,NA,⋯,38,4227359248,23.2,27.6,28.2,21.0,0.0,32.4,13.0,http://duffel.rail.bio/recount3/human/data_sources/gtex/base_sums/OD/BLOOD/KY/gtex.base_sums.BLOOD_GTEX-QMR6-0005-SM-32PKY.1.ALL.bw
10044936,GTEX-POMQ-0006-SM-33HBY.1,BLOOD,SRR661854,GTEX-POMQ,2,20-29,0,GTEX-POMQ-0006-SM-33HBY,NA,⋯,38,4644711576,27.4,26.6,24.3,21.7,0.0,31.0,11.4,http://duffel.rail.bio/recount3/human/data_sources/gtex/base_sums/OD/BLOOD/BY/gtex.base_sums.BLOOD_GTEX-POMQ-0006-SM-33HBY.1.ALL.bw
10044938,GTEX-S32W-0004-SM-2XCE9.1,BLOOD,SRR662174,GTEX-S32W,2,50-59,0,GTEX-S32W-0004-SM-2XCE9,NA,⋯,38,3508083924,25.3,25.0,24.6,25.0,0.0,33.8,14.5,http://duffel.rail.bio/recount3/human/data_sources/gtex/base_sums/OD/BLOOD/E9/gtex.base_sums.BLOOD_GTEX-S32W-0004-SM-2XCE9.1.ALL.bw
10044940,GTEX-U3ZN-0006-SM-3DB7Y.1,BLOOD,SRR662847,GTEX-U3ZN,2,30-39,0,GTEX-U3ZN-0006-SM-3DB7Y,NA,⋯,38,5086613500,26.5,26.8,24.0,22.7,0.0,31.0,11.5,http://duffel.rail.bio/recount3/human/data_sources/gtex/base_sums/OD/BLOOD/7Y/gtex.base_sums.BLOOD_GTEX-U3ZN-0006-SM-3DB7Y.1.ALL.bw
10044963,GTEX-WFON-0005-SM-3NMC9.1,BLOOD,SRR816745,GTEX-WFON,1,40-49,0,GTEX-WFON-0005-SM-3NMC9,NA,⋯,38,4670368644,21.6,28.2,29.1,21.0,0.1,32.7,13.5,http://duffel.rail.bio/recount3/human/data_sources/gtex/base_sums/OD/BLOOD/C9/gtex.base_sums.BLOOD_GTEX-WFON-0005-SM-3NMC9.1.ALL.bw


In [66]:
colnames(colData(rse_gene_sra_SRP166108))

[1] "rail_id"                                                           
  [2] "external_id"                                                       
  [3] "study"                                                             
  [4] "sra.sample_acc.x"                                                  
  [5] "sra.experiment_acc"                                                
  [6] "sra.submission_acc"                                                
  [7] "sra.submission_center"                                             
  [8] "sra.submission_lab"                                                
  [9] "sra.study_title"                                                   
 [10] "sra.study_abstract"                                                
 [11] "sra.study_description"                                             
 [12] "sra.experiment_title"                                              
 [13] "sra.design_description"                                            
 [14] "sra.sample_description"                                            
 [15] "sra.library_name"                                                  
 [16] "sra.library_strategy"                                              
 [17] "sra.library_source"                                                
 [18] "sra.library_selection"                                             
 [19] "sra.library_layout"                                                
 [20] "sra.paired_nominal_length"                                         
 [21] "sra.paired_nominal_stdev"                                          
 [22] "sra.library_construction_protocol"                                 
 [23] "sra.platform_model"                                                
 [24] "sra.sample_attributes"                                             
 [25] "sra.experiment_attributes"                                         
 [26] "sra.spot_length"                                                   
 [27] "sra.sample_name"                                                   
 [28] "sra.sample_title"                                                  
 [29] "sra.sample_bases"                                                  
 [30] "sra.sample_spots"                                                  
 [31] "sra.run_published"                                                 
 [32] "sra.size"                                                          
 [33] "sra.run_total_bases"                                               
 [34] "sra.run_total_spots"                                               
 [35] "sra.num_reads"                                                     
 [36] "sra.num_spots"                                                     
 [37] "sra.read_info"                                                     
 [38] "sra.run_alias"                                                     
 [39] "sra.run_center_name"                                               
 [40] "sra.run_broker_name"                                               
 [41] "sra.run_center"                                                    
 [42] "recount_project.project"                                           
 [43] "recount_project.organism"                                          
 [44] "recount_project.file_source"                                       
 [45] "recount_project.metadata_source"                                   
 [46] "recount_project.date_processed"                                    
 [47] "recount_qc.aligned_reads%.chrm"                                    
 [48] "recount_qc.aligned_reads%.chrx"                                    
 [49] "recount_qc.aligned_reads%.chry"                                    
 [50] "recount_qc.bc_auc.all_reads_all_bases"                             
 [51] "recount_qc.bc_auc.all_reads_annotated_bases"                       
 [52] "recount_qc.bc_auc.unique_reads_all_bases"                          
 [53] "recount_qc.bc_auc.unique_reads_annotated_bases"                    
 [54] "recount_qc.bc_auc.al

In [63]:
colData(rse_gene_sra_SRP166108) |> 
    as_tibble() %>% 
    select(contains("age")) %>%
    select(!contains("average")) %>%
    select(!contains("coverage")) %>%
    select(!contains("stage")) %>%
    head()

1
2
3
4
5
6

In [54]:
colData(rse_gene_tcga_gbm) |> 
    as_tibble() %>% 
    select(contains("age")) %>%
    select(!contains("average")) %>%
    select(!contains("coverage")) %>%
    select(!contains("stage")) %>%
    head()

tcga.gdc_cases.diagnoses.age_at_diagnosis,tcga.cgc_case_age_at_diagnosis,tcga.xml_primary_pathology_age_at_initial_pathologic_diagnosis,tcga.xml_age_at_initial_pathologic_diagnosis,tcga.xml_asbestos_exposure_age,tcga.xml_asbestos_exposure_age_last,tcga.xml_birth_control_pill_history_usage_category,tcga.xml_age_began_smoking_in_years,tcga.xml_er_level_cell_percentage_category,tcga.xml_history_of_esophageal_cancer,⋯,tcga.xml_cumulative_agent_total_dose,tcga.xml_hydroxyurea_agent_administered_day_count,tcga.xml_person_history_nonmedical_leukemia_causing_agent_type,tcga.xml_lab_procedure_blast_cell_outcome_percentage_value,tcga.xml_prior_tamoxifen_administered_usage_category,tcga.xml_radiosensitizing_agent_administered_indicator,tcga.xml_first_diagnosis_age_asth_ecz_hay_fev_mold_dust,tcga.xml_first_diagnosis_age_of_food_allergy,tcga.xml_first_diagnosis_age_of_animal_insect_allergy,tcga.xml_undescended_testis_corrected_age
<int>,<int>,<lgl>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
26796,73,NA,73,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
21384,58,NA,58,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
19053,52,NA,52,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
23343,63,NA,63,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
20159,55,NA,55,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
17880,48,NA,48,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [67]:
colData(rse_gene_gtex_blood) |> 
    as_tibble() %>% 
    select(contains("age")) %>%
    select(!contains("average")) %>%
    select(!contains("coverage")) %>%
    distinct()

gtex.age
<chr>
40-49
50-59
20-29
30-39
60-69
70-79


# Count normalization

In [8]:
assay(rse_gene_gtex_blood) |> head()

,GTEX-R55C-0005-SM-3GAE9.1,GTEX-QMR6-0005-SM-32PKY.1,GTEX-POMQ-0006-SM-33HBY.1,GTEX-S32W-0004-SM-2XCE9.1,GTEX-U3ZN-0006-SM-3DB7Y.1,GTEX-WFON-0005-SM-3NMC9.1,GTEX-1399R-0002-SM-62LD9.1,GTEX-14PJ4-0006-SM-7RHFY.1,GTEX-14PJO-0005-SM-9JGGY.1,GTEX-15SDE-0006-SM-7PC19.1,⋯,GTEX-1I19N-0006-SM-A9G18.1,GTEX-1I1GT-0003-SM-EZ6M8.1,GTEX-1I1HK-0005-SM-ARZL8.1,GTEX-1IDFM-0005-SM-ADEGX.1,GTEX-1J8JJ-0006-SM-CNNOX.1,GTEX-1JMPY-0006-SM-DHXJ8.1,GTEX-1K2DU-0005-SM-DIPE8.1,GTEX-1KANC-0006-SM-9YFL8.1,GTEX-1NSGN-0005-SM-DKPPX.1,GTEX-1QP29-0005-SM-DLZQX.1
ENSG00000278704.1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000277400.1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000274847.1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000277428.1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000276256.1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000278198.1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [9]:
assay(rse_gene_gtex_blood) |> str()

 num [1:63856, 1:1048] 0 0 0 0 0 0 0 0 0 0 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:63856] "ENSG00000278704.1" "ENSG00000277400.1" "ENSG00000274847.1" "ENSG00000277428.1" ...
  ..$ : chr [1:1048] "GTEX-R55C-0005-SM-3GAE9.1" "GTEX-QMR6-0005-SM-32PKY.1" "GTEX-POMQ-0006-SM-33HBY.1" "GTEX-S32W-0004-SM-2XCE9.1" ...


In [10]:
tbl_blood <- assay(rse_gene_gtex_blood) |>
    as.data.frame() |>
    rownames_to_column(var = "gene") %>%
    as_tibble()
tbl_blood %>% head()

gene,GTEX-R55C-0005-SM-3GAE9.1,GTEX-QMR6-0005-SM-32PKY.1,GTEX-POMQ-0006-SM-33HBY.1,GTEX-S32W-0004-SM-2XCE9.1,GTEX-U3ZN-0006-SM-3DB7Y.1,GTEX-WFON-0005-SM-3NMC9.1,GTEX-1399R-0002-SM-62LD9.1,GTEX-14PJ4-0006-SM-7RHFY.1,GTEX-14PJO-0005-SM-9JGGY.1,⋯,GTEX-1I19N-0006-SM-A9G18.1,GTEX-1I1GT-0003-SM-EZ6M8.1,GTEX-1I1HK-0005-SM-ARZL8.1,GTEX-1IDFM-0005-SM-ADEGX.1,GTEX-1J8JJ-0006-SM-CNNOX.1,GTEX-1JMPY-0006-SM-DHXJ8.1,GTEX-1K2DU-0005-SM-DIPE8.1,GTEX-1KANC-0006-SM-9YFL8.1,GTEX-1NSGN-0005-SM-DKPPX.1,GTEX-1QP29-0005-SM-DLZQX.1
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000278704.1,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000277400.1,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000274847.1,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000277428.1,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000276256.1,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000278198.1,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000276932.1,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000273496.1,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000273735.4,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [12]:
genes <- sapply(tbl_blood$gene, function(x) strsplit(x, "[.]")[[1]][1])
genes[1:10]

ENSG00000278704.1 ENSG00000277400.1 ENSG00000274847.1 ENSG00000277428.1 
"ENSG00000278704" "ENSG00000277400" "ENSG00000274847" "ENSG00000277428" 
ENSG00000276256.1 ENSG00000278198.1 ENSG00000276932.1 ENSG00000273496.1 
"ENSG00000276256" "ENSG00000278198" "ENSG00000276932" "ENSG00000273496" 
ENSG00000273735.4 ENSG00000273517.1 
"ENSG00000273735" "ENSG00000273517"

In [13]:
listFilters(mart) %>% filter(str_detect(name, "hgnc"))

name,description
<chr>,<chr>
with_hgnc,With HGNC Symbol ID(s)
with_hgnc_trans_name,With Transcript name ID(s)
hgnc_id,HGNC ID(s) [e.g. HGNC:100]
hgnc_symbol,HGNC symbol(s) [e.g. A1BG]
hgnc_trans_name,Transcript name ID(s) [e.g. A1BG-201]


In [14]:
class(1:10)

[1] "integer"

In [16]:
gene_IDs <- getBM(filters=c("ensembl_gene_id"), attributes= c("ensembl_gene_id","hgnc_symbol"),
              values = genes, mart= mart)

In [26]:
gene_IDs |> head()

,ensembl_gene_id,hgnc_symbol
,<chr>,<chr>
1,ENSG00000056678,KIFC1
2,ENSG00000096150,RPS18
3,ENSG00000096155,BAG6
4,ENSG00000096171,VARS1
5,ENSG00000111971,LY6G5C
6,ENSG00000112459,OR14J1


In [19]:
tbl_blood$gene <- gene_IDs[match(substr(tbl_blood$gene, 1, 15), gene_IDs$ensembl_gene_id), "hgnc_symbol"]

In [20]:
is.na(tbl_blood$gene) |> table()


FALSE  TRUE 
63102   754 

In [21]:
tbl_blood <- tbl_blood %>% filter(gene != "")

In [28]:
tbl_blood %>% tail()

gene,GTEX-R55C-0005-SM-3GAE9.1,GTEX-QMR6-0005-SM-32PKY.1,GTEX-POMQ-0006-SM-33HBY.1,GTEX-S32W-0004-SM-2XCE9.1,GTEX-U3ZN-0006-SM-3DB7Y.1,GTEX-WFON-0005-SM-3NMC9.1,GTEX-1399R-0002-SM-62LD9.1,GTEX-14PJ4-0006-SM-7RHFY.1,GTEX-14PJO-0005-SM-9JGGY.1,⋯,GTEX-1I19N-0006-SM-A9G18.1,GTEX-1I1GT-0003-SM-EZ6M8.1,GTEX-1I1HK-0005-SM-ARZL8.1,GTEX-1IDFM-0005-SM-ADEGX.1,GTEX-1J8JJ-0006-SM-CNNOX.1,GTEX-1JMPY-0006-SM-DHXJ8.1,GTEX-1K2DU-0005-SM-DIPE8.1,GTEX-1KANC-0006-SM-9YFL8.1,GTEX-1NSGN-0005-SM-DKPPX.1,GTEX-1QP29-0005-SM-DLZQX.1
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ELOCP24,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
TRPC6P,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
IL9R,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
WASIR1,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
WASH6P,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
DDX11L16,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
